In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import re
import string

In [2]:
df_True=pd.read_csv('True.csv')
df_Fake=pd.read_csv('Fake.csv')

In [3]:
df_True.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
df_Fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df_True['class']=1
df_Fake['class']=0

In [6]:
df_True.shape,df_Fake.shape

((21417, 5), (23481, 5))

In [7]:
df_merge=pd.concat([df_True,df_Fake],axis=0)

In [8]:
df_merge = df_merge.reset_index(drop=True)

In [9]:
df_merge

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [10]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   class    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [11]:
df_merge.isnull().sum()

title      0
text       0
subject    0
date       0
class      0
dtype: int64

In [12]:
df_merge.duplicated().sum()

209

In [13]:
df_merge.drop_duplicates(keep='first',inplace=True)

In [14]:
df_merge.duplicated().sum()

0

In [15]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [16]:
df_merge.drop(columns=['title','subject','date'],axis=1,inplace=True)

In [17]:
df_merge

,text,class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...
44893,21st Century Wire says As 21WIRE reported earl...,0
44894,21st Century Wire says It s a familiar theme. ...,0
44895,Patrick Henningsen 21st Century WireRemember ...,0
44896,21st Century Wire says Al Jazeera America will...,0


In [18]:
shuffled_data=df_merge.sample(frac=1)

In [19]:
shuffled_data.head()

,text,class
24287,"On Tuesday, it was announced that Donald Trump...",0
17314,"ISLAMABAD (Reuters) - Afghanistan, China, Paki...",1
37062,The Friday document dump today included 300 of...,0
21089,HONG KONG (Reuters) - Hong Kong s public broad...,1
33077,https://www.youtube.com/watch?time_continue=1&...,0


In [20]:
def wordopt(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\W', ' ', text)  # Replace non-word characters with spaces
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    # Consider adding steps for contractions, stop words, stemming/lemmatization
    return text

In [21]:
shuffled_data['text']=shuffled_data['text'].apply(wordopt)

In [22]:
x=shuffled_data['text']
y=shuffled_data['class']

In [23]:
x

24287    on tuesday  it was announced that donald trump...
17314    islamabad  reuters    afghanistan  china  paki...
37062    the friday document dump today included 300 of...
21089    hong kong  reuters    hong kong s public broad...
33077    https   www youtube com watch time_continue 1 ...
                               ...                        
23838    anyone who respects democracy and the way this...
8477      reuters    a former north carolina staffer is...
17797    moscow  reuters    russia accused the united s...
4667      oslo  reuters    a 2015 global pact for fight...
40443    hey mainstream media when are you going to tak...
Name: text, Length: 44689, dtype: object

In [24]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [25]:
x_train.shape,x_test.shape

((31282,), (13407,))

In [26]:
#Feature engineering: Creates numerical representations of text data that can be used by machine learning algorithms.
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

In [27]:
xv_train.shape,xv_test.shape,y_train.shape

((31282, 104157), (13407, 104157), (31282,))

In [28]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [29]:
y_pred=lr.predict(xv_test)

In [30]:
lr.score(xv_test,y_test)#accuracy score

0.9879913478033863

In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7089
           1       0.99      0.99      0.99      6318

    accuracy                           0.99     13407
   macro avg       0.99      0.99      0.99     13407
weighted avg       0.99      0.99      0.99     13407



In [32]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(xv_train,y_train)
y_pred=dt.predict(xv_test)

In [33]:
dt.score(xv_test,y_test)

0.9952263742820915

In [34]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7089
           1       1.00      0.99      0.99      6318

    accuracy                           1.00     13407
   macro avg       1.00      1.00      1.00     13407
weighted avg       1.00      1.00      1.00     13407



In [35]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(xv_train,y_train)
y_pred=gb.predict(xv_test)

In [ ]:
gb.score(xv_test,y_test)

: 

In [ ]:
print(classification_report(y_test,y_pred))

: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(xv_train,y_train)
y_pred=rf.predict(xv_test)

: 

In [ ]:
rf.score(xv_test,y_test)

: 

In [ ]:
print(classification_report(y_test,y_pred))

: 

## Selecting 2 best models and making a pickle file

In [ ]:
import pickle
pickle.dump(dt,open('DecisionTreeClassifier.pkl','wb'))
pickle.dump(gb,open('GradientBoostingClassifier.pkl','wb'))
pickle.dump(vectorization,open('vectorization.pkl','wb'))

: 

In [ ]:
def output_label(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "Not a fake News"
def manual_testing(news):
    testing_news={"text":[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"]=new_def_test["text"].apply(wordopt)
    new_x_test=new_def_test["text"]
    new_xv_test=vectorization.transform(new_x_test)
    pred_lr=lr.predict(new_xv_test)
    pred_dt=dt.predict(new_xv_test)
    pred_gb=gb.predict(new_xv_test)
    pred_rf=rf.predict(new_xv_test)
    return print("\n\n LR Prediction:{} \n DT Prediction:{} \n GBC Prediction:{} \n RFC Prediction:{}".format(output_label(pred_lr[0]),
                                                                                                             output_label(pred_dt[0]),
                                                                                                             output_label(pred_gb[0]),
                                                                                                             output_label(pred_rf[0])))

: 

In [ ]:
news=str(input())
manual_testing(news)

: 